### Trabalho Final - Aprendizado de Máquina
- Vinícius Braga Freire - 2020054889

#### 0. Introdução

Este projeto...

#### 1. Processamento de dados

Assim, precisamos primeiro extrair e tratar os dados relevantes da nossa base de dados de forma que consigamos aprender sob esses.

##### 1.1 Interações

O primeiro passo será extrair dados que nos permitam observar as interações entre funcionários e\nportante\ncomo as equipes são estruturadas. Dada a chegada de novos dados do projeto IMAGINE (um acrônimo interno da empresa DEXCO), foi reformulada a maneira como iremos modelar e interpretar times neste projeto.

O projeto IMAGINE permite que funcionários façam sugestões de projetos internos que sejam inovadores. Funcionários que apresentem ideias boas têm seus projetos aceitos e implantados e, portanto, recebem um bônus em remuneração por tal feito.

Desta forma, nosso dado irá nos informar o status (aceito, rejeitado, em processo de implantação, em espera de avaliação) de projetos que foram sugeridos e por quais funcionários.

Observe que mais de um funcionário podem trabalhar juntos para sugerirem ideias de projetos. É a partir deste fato que iremos construir um novo dado seguindo as seguintes regras:

- Em projetos aceitos e que foram sugeridos por mais de 1 pessoa, devemos sinalizar positivamente que as pessoas envolvidas `DEVEM` trabalhar em equipe. Assim
essas pessoas 2 a 2 (cada par possível dentre esse conjunto de pessoas) tem um aval positivo\ndado que o projeto sugerido por eles foi aceito como uma boa ideia.
- Em projetos rejeitados e que foram sugeridos por mais de 1 pessoa, devemos sinalizar negativamente que as pessoas envolvidas `NÃO DEVEM` trabalhar em equipe.

In [242]:
import pandas as pd
import numpy as np

projetos_df = pd.read_csv("./data/in/imagine.csv", encoding="utf-8", sep=",")
print(projetos_df.info())
projetos_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58730 entries, 0 to 58729
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   name                  58716 non-null  object
 1   Matricula             58730 non-null  int64 
 2   status                58730 non-null  object
 3   local                 58729 non-null  object
 4   area                  58729 non-null  object
 5   data                  58730 non-null  object
 6   suggestionsRegarding  27664 non-null  object
dtypes: int64(1), object(6)
memory usage: 3.1+ MB
None


,name,Matricula,status,local,area,data,suggestionsRegarding
0,Custometro,10077261,Fila de execução,ADM CENTRAL,CSS-01 - PRESIDENCIA CONSELHO ADM - EC,2/23/2022,NaN
1,otmizar a temperatura do oléo térmico no desca...,10023570,Fila de execução,FÁB ITAPETININGA,DXCAVAPO61 - Caldeira de Vapor,7/5/2022,Produtividade
2,limpeza de sensor de nivel,10023570,Concluída,FÁB ITAPETININGA,DXCAVAPO61 - Caldeira de Vapor,10/20/2022,NaN
3,Segurança sempre!,10023570,Fila de execução,FÁB ITAPETININGA,MDP - Caldeira,11/9/2021,Segurança
4,Em busca da economia do BPF!,10023570,Fila de execução,FÁB ITAPETININGA,MII-02 - CALDEIRA E UILIDADES - IT,8/22/2019,Redução de Custos


Observe que temos alguns dados faltantes, principalmente na coluna `Name`, que será importante para nós agruparmos os dados por projeto para acharmos as equipes. Assim, vamos limpar os dados.

In [243]:
projetos = projetos_df[['name', 'Matricula', 'status']]
# Remove null names since it's impossible to group the data
projetos = projetos.dropna(subset=['name'])
projetos

,name,Matricula,status
0,Custometro,10077261,Fila de execução
1,otmizar a temperatura do oléo térmico no desca...,10023570,Fila de execução
2,limpeza de sensor de nivel,10023570,Concluída
3,Segurança sempre!,10023570,Fila de execução
4,Em busca da economia do BPF!,10023570,Fila de execução
...,...,...,...
58725,PONTO DE COLETA SELETIVA!,10001462,Não qualificada
58726,limpou a agua,10001131,Não qualificada
58727,Nacionalização de segmento do desfibrador,10001131,Fila de execução
58728,Redução Mão de Obra Terceirizada na Colômbia,27534,Fila de execução


Vamos codificar o `status` em números para que possamos processar os dados posteriormente de maneira mais rápida. O mapeamento será o seguinte:
- Não qualificada = 0
- Concluída = 1
- Fila de execução = 2
- Aguardando qualificação = Esperando Calificación = 3
- Em execução = 4
- Stand By = 5
- Cancelada = 6
- Pendente de patente = 7

In [244]:
NUM_TO_STATS = ['Não qualificada',
                'Concluída',
                'Fila de execução', 
                'Aguardando qualificação',
                'Em execução',
                'Stand By', 
                'Cancelada',
                'Pendente de patente']
STATS_TO_NUM = {'Não qualificada': 0,
                'Concluída': 1,
                'Fila de execução': 2, 
                'Aguardando qualificação': 3,
                'Esperando Calificación': 3,
                'Em execução': 4,
                'Stand By': 5, 
                'Cancelada': 6,
                'Pendente de patente': 7}

print(projetos['status'].unique())
# Transform the data
projetos['status'] =  projetos['status'].apply(lambda row: STATS_TO_NUM[row]).values
projetos

['Fila de execução' 'Concluída' 'Aguardando qualificação' 'Em execução'
 'Stand By' 'Não qualificada' 'Cancelada' 'Pendente de patente'
 'Esperando Calificación']


,name,Matricula,status
0,Custometro,10077261,2
1,otmizar a temperatura do oléo térmico no desca...,10023570,2
2,limpeza de sensor de nivel,10023570,1
3,Segurança sempre!,10023570,2
4,Em busca da economia do BPF!,10023570,2
...,...,...,...
58725,PONTO DE COLETA SELETIVA!,10001462,0
58726,limpou a agua,10001131,0
58727,Nacionalização de segmento do desfibrador,10001131,2
58728,Redução Mão de Obra Terceirizada na Colômbia,27534,2


Agora vamos agrupar as pessoas que trabalharam no mesmo projeto, garatindo que uma pessoa só apareça uma vez por projeto e que equipes de apenas uma pessoa sejam removidas, de forma a posteriomente obtermos todas as equipes.

In [245]:
# Group people that worked in the same project
mask = projetos.groupby('name').count()['Matricula'] > 1
# Removes projects that only one person work on
map = {}
for name, valid in mask.items(): map[name] = valid
full_mask = projetos['name'].apply(lambda x: map[x] if x in map else print('ERROR AT' + x))
proj_gt_1 = projetos[full_mask]

# Avoids repeated person in same project
proj_gt_1 = proj_gt_1.drop_duplicates()
proj_gt_1

,name,Matricula,status
0,Custometro,10077261,2
1,otmizar a temperatura do oléo térmico no desca...,10023570,2
9,Marcação de peças,10077515,3
10,GABARITOS PARA TODAS AS BANCAS,10079166,2
11,reaproveitamento térmico para climatização amb...,10073270,3
...,...,...,...
58725,PONTO DE COLETA SELETIVA!,10001462,0
58726,limpou a agua,10001131,0
58727,Nacionalização de segmento do desfibrador,10001131,2
58728,Redução Mão de Obra Terceirizada na Colômbia,27534,2


In [227]:
gp = proj_gt_1.groupby('name')
final_df = pd.DataFrame()

def statusHierarchy(status, past):
    return 0;

# For each project, removes internal redundancy and put it in a final data frame
for name, workers in gp:
    workers_map = {}
    curr_state = 0;
    
    def internal_mapping(x):
        curr_mat = x['Matricula'];
        # If this worker has already been registred
        if curr_mat in workers_map:
            workers_map[curr_mat] = statusHierarchy(x['status'], workers_map[curr_mat])
        else:
            workers_map[curr_mat] = x['status']
        curr_state = statusHierarchy(workers_map[curr_mat], curr_state)
        
    
    a = workers.apply(lambda x: internal_mapping(x))
    
    print(a)
    break;
gp

name
Matricula
status
                                       name  Matricula  \
14874   - Incentivos e recompensas à equipe   10087417   
14875   - Incentivos e recompensas à equipe   10087417   

                        status  
14874  Aguardando qualificação  
14875         Fila de execução  


##### 1.2. Infos Pessoais

QAAAA

In [ ]:
import pandas as pd

infos_pessoas_df = pd.read_csv("./data/in/infos_pessoas.csv", encoding="cp860")
infos_pessoas_df.head()

#### 2. Treinamento de modelo

#### 3. Resultado e validação de modelo